In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
import os

In [86]:
def open_datasets(model, experiment, runs):
    dirpath = f"/datastore/{model}/{experiment}"
    ds_list = []
    if model in ['cam5', 'miroc5']:
        dirpath = os.path.join(dirpath, '*_run{:03d}_*.nc')
    elif model=='hadgem3':
        dirpath = os.path.join(dirpath, '*_r1i1p{}_*.nc')
    else:
        raise ValueError(f"Model '{model}' not found")
    for r in runs:
        ds_list.append(xr.open_mfdataset(dirpath.format(r)).expand_dims({'run':[r]}))
    ds = xr.concat(ds_list, dim='run')
    return ds

In [ ]:
experiment='nat_hist'
runs=[1,2]

In [26]:
ds_cam5 = open_datasets('cam5', experiment, runs)
ds_cam5

,Array,Chunk
Bytes,1.32 MB,11.68 kB
Shape,"(2, 20621, 2, 2)","(1, 365, 2, 2)"
Count,616 Tasks,114 Chunks
Type,object,numpy.ndarray
,Array,Chunk
Bytes,9.12 GB,80.73 MB
Shape,"(2, 20621, 1, 192, 288)","(1, 365, 1, 192, 288)"
Count,524 Tasks,114 Chunks
Type,float32,numpy.ndarray


In [27]:
ds_had = open_datasets('hadgem3', experiment, runs)
ds_had

<xarray.Dataset>
Dimensions:    (bnds: 2, height: 1, lat: 324, lon: 432, run: 2, time: 19440)
Coordinates:
  * lon        (lon) float32 0.41666666 1.25 2.0833333 ... 358.75 359.5833
  * lat        (lat) float32 -89.72222 -89.166664 ... 89.16667 89.72223
  * time       (time) object 1960-01-01 12:00:00 ... 2013-12-30 12:00:00
  * height     (height) float32 2.0
  * run        (run) int64 1 2
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (run, time, lat, bnds) float32 dask.array<chunksize=(1, 3600, 324, 2), meta=np.ndarray>
    lon_bnds   (run, time, lon, bnds) float32 dask.array<chunksize=(1, 3600, 432, 2), meta=np.ndarray>
    time_bnds  (run, time, bnds) object dask.array<chunksize=(1, 3600, 2), meta=np.ndarray>
    tas        (run, time, height, lat, lon) float32 dask.array<chunksize=(1, 3600, 1, 324, 432), meta=np.ndarray>
Attributes:
    history:                   2016-03-25T02:11:25Z: Naming and attribute con...
    source:                    MOHC Unified Model [N216L85]
    institution:               Met Office Hadley Centre, Fitzroy Road, Exeter...
    institute_id:              MOHC
    model_id:                  HadGEM3-A-N216
    forcing:                   GHG, Oz, LU, Sl, Vl, AA, (GHG = CO2, N2O, CH4,...
    parent_experiment_id:      N/A
    parent_experiment_rip:     N/A
    branch_time:               0.0
    contact:                   peter.stott@metoffice.gov.uk, andrew.ciavarell...
    initialization_method:     1
    physics_version:           1
    tracking_id:               b362115c-ed6c-4beb-ab8d-664c7e9d7519
    mo_runid:                  aorsa
    product:                   output
    experiment:                CMIP5-est1
    frequency:                 day
    creation_date:             2016-03-25T02:11:25Z
    project_id:                EUCLEIA, and C20C+ Detection and Attribution P...
    table_id:                  Table day (13 January 2015) 159846346d11cd9e5d...
    title:                     HadGEM3-A-N216 model output prepared for EUCLE...
    parent_experiment:         N/A
    modeling_realm:            atmos
    realization:               1
    cmor_version:              2.9.1
    experiment_family:         Nat-Hist
    parent_experiment_family:  N/A
    orig_experiment_id:        historicalNat
    subexperiment:             v1-0
    parent_subexperiment:      N/A
    orig_project_id:           EUCLEIA
    CDO:                       Climate Data Operators version 1.7.0 (http://m...
    run_id:                    r1i1p1
    acknowledgement:           This simulation was performed under funding fr...
    parent_run_id:             N/A
    license:                   I agree to restrict my use of EUCLEIA model ou...

Try some different times to see if xarray is doing any caching behind the scenes

In [100]:
r=0
%time _ = ds_had.isel(time=0, run=r).load()
%time _ = ds_had.isel(time=1, run=r).load()
%time _ = ds_had.isel(time=600, run=r).load()
%time _ = ds_had.isel(time=601, run=r).load()

CPU times: user 1.56 s, sys: 1.64 s, total: 3.2 s
Wall time: 3.18 s
CPU times: user 1.51 s, sys: 1.65 s, total: 3.16 s
Wall time: 3.15 s
CPU times: user 1.52 s, sys: 1.64 s, total: 3.17 s
Wall time: 3.15 s
CPU times: user 1.52 s, sys: 1.63 s, total: 3.15 s
Wall time: 3.13 s


In [101]:
r=1
%time _ = ds_cam5.isel(time=0, run=r).load()
%time _ = ds_cam5.isel(time=1, run=r).load()
%time _ = ds_cam5.isel(time=600, run=r).load()
%time _ = ds_cam5.isel(time=601, run=r).load()

CPU times: user 4.4 s, sys: 10.8 s, total: 15.2 s
Wall time: 15.4 s
CPU times: user 4.12 s, sys: 9.44 s, total: 13.6 s
Wall time: 13.6 s
CPU times: user 4.44 s, sys: 6.96 s, total: 11.4 s
Wall time: 11.4 s
CPU times: user 4.47 s, sys: 10.1 s, total: 14.6 s
Wall time: 14.7 s


In [102]:
%time _ = ds_had.isel(time=slice(0,360), run=1).load()
%time _ = ds_cam5.isel(time=slice(0,360), run=1).load()

CPU times: user 1.6 s, sys: 1.59 s, total: 3.19 s
Wall time: 3.18 s
CPU times: user 4.42 s, sys: 6.9 s, total: 11.3 s
Wall time: 11.3 s


So to summarise the above. It takes ~3 or ~11 seconds to load in a full years worth of data or the same time for a single day due to the way the data is stored. I think that ideally we would sample randomly rather than load in a full year's worth of data and sample from that. If we sampled one year at a time then we'd likely overtrain to specific predefined SSTs in each section. Therefore we need to restructure the files to allow faster loading of randomly sampled data.

## zarr vs Netcdf speed test

I have now made a zarr copy of the miroc5 data. Lets do a load speed test

In [89]:
runs = np.arange(1,11)
experiment = 'nat_hist'

In [ ]:
ds_miroc = open_datasets('miroc5', experiment, runs)

In [92]:
dz_miroc = xr.open_zarr("/datastore/miroc5/nat_hist_zarr", consolidated=True)

In [95]:
r=0
%timeit -n 1 -r 10 ds_miroc.isel(time=0, run=r).load()
%timeit -n 1 -r 10 dz_miroc.isel(time=0, run=r).load()

525 ms ± 3.96 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
15.3 ms ± 1.22 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


zarr without compression is ~35 times faster so lets go wth it